# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [32]:
site_url = "https://dev.drosera.io/"
site = Website(site_url)
site.links

['/#vocs-content',
 '/introduction',
 'https://github.com/drosera-network/examples',
 '/introduction',
 'https://github.com/drosera-network/examples',
 '/introduction',
 'https://github.com/drosera-network',
 '/trappers/getting-started',
 '/operators/installation',
 '/litepaper',
 '/use-cases',
 '/#common-actions',
 '/#leveraging-the-best',
 'https://github.com/risc0',
 'https://github.com/foundry-rs/foundry',
 'https://github.com/bluealloy/revm',
 'https://github.com/libp2p',
 '/#features',
 'https://x.com/DroseraNetwork/status/1796210361940504956',
 'https://risczero.com/',
 'https://soliditylang.org/',
 'https://getfoundry.sh/',
 'https://www.terraform.io/',
 'https://libp2p.io/',
 '/#community',
 'https://discord.gg/drosera',
 'https://github.com/drosera-network/examples/issues',
 '/#credit',
 'https://github.com/drosera-network',
 'https://discord.gg/drosera',
 'https://x.com/droseranetwork',
 'https://t.me/droseraofficial']

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [33]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about a blockchain protocol, \
such as links to an About page, or dev docs, or litepaper pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "use cases page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [34]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about a blockchain protocol, such as links to an About page, or dev docs, or litepaper pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "use cases page": "url": "https://another.full.url/careers"}
    ]
}



In [42]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the blockchain protocol, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [44]:
print(get_links_user_prompt(site))

Here is the list of links on the website of https://dev.drosera.io/ - please decide which of these are relevant web links for a brochure about the blockchain protocol, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/#vocs-content
/introduction
https://github.com/drosera-network/examples
/introduction
https://github.com/drosera-network/examples
/introduction
https://github.com/drosera-network
/trappers/getting-started
/operators/installation
/litepaper
/use-cases
/#common-actions
/#leveraging-the-best
https://github.com/risc0
https://github.com/foundry-rs/foundry
https://github.com/bluealloy/revm
https://github.com/libp2p
/#features
https://x.com/DroseraNetwork/status/1796210361940504956
https://risczero.com/
https://soliditylang.org/
https://getfoundry.sh/
https://www.terraform.io/
https://libp2p.io/
/#community
https://discord.gg/drosera
https://github.com/drosera-network/examples/issues
/#cr

In [45]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

In [46]:
get_links(site_url)

{'links': [{'type': 'introduction page',
   'url': 'https://dev.drosera.io/introduction'},
  {'type': 'litepaper', 'url': 'https://dev.drosera.io/litepaper'},
  {'type': 'use cases page', 'url': 'https://dev.drosera.io/use-cases'},
  {'type': 'getting started guide',
   'url': 'https://dev.drosera.io/trappers/getting-started'},
  {'type': 'installation guide',
   'url': 'https://dev.drosera.io/operators/installation'},
  {'type': 'GitHub repository', 'url': 'https://github.com/drosera-network'},
  {'type': 'GitHub examples',
   'url': 'https://github.com/drosera-network/examples'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [47]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [48]:
print(get_all_details(site_url))

Found links: {'links': [{'type': 'introduction page', 'url': 'https://dev.drosera.io/introduction'}, {'type': 'github repo', 'url': 'https://github.com/drosera-network'}, {'type': 'github examples', 'url': 'https://github.com/drosera-network/examples'}, {'type': 'getting started guide', 'url': 'https://dev.drosera.io/trappers/getting-started'}, {'type': 'installation guide', 'url': 'https://dev.drosera.io/operators/installation'}, {'type': 'litepaper', 'url': 'https://dev.drosera.io/litepaper'}, {'type': 'use cases page', 'url': 'https://dev.drosera.io/use-cases'}, {'type': 'community channel', 'url': 'https://discord.gg/drosera'}]}
Landing page:
Webpage Title:
Drosera · Ethereum Evolved – Docs
Webpage Contents:
Skip to content
Search...
Docs
Examples
1.0.0
Docs
Examples
1.0.0
Menu
Drosera · Ethereum Evolved
Drosera
Infinite bandwidth for smart contracts using
decentralized
,
verifiable
, and
composable
infrastructure
Introduction
GitHub
Terminal
bash
Copy
curl
-L
https://app.drosera.i

In [70]:
system_prompt = "You are the famous Andrew Tate (millionaire, playboy, kick boxer) and you will analyze the contents of several relevant pages from a blockchain protocol website/developer docs site \
and creates a short brochure about the protocol for prospective users, investors and recruits. Respond in markdown.\
Include details of protocol unique design, necessity in the market, high-level technical design, use-case, and development flow, if you have the information. Respond in the strong and unmistakable style of Andrew Tate. Add some classic Andrew Tate style shit talking in your response, even if it isn't super relevant to the protocol you're reviewing"

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [71]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a blockchain protocol called: {company_name}\n"
    user_prompt += f"Here are the contents of its developer docs and other relevant pages; use this information to build a short brochure of the protocol in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 5,000 characters
    return user_prompt

In [59]:
get_brochure_user_prompt("Drosera", site_url)

Found links: {'links': [{'type': 'introduction page', 'url': 'https://dev.drosera.io/introduction'}, {'type': 'litepaper page', 'url': 'https://dev.drosera.io/litepaper'}, {'type': 'use cases page', 'url': 'https://dev.drosera.io/use-cases'}, {'type': 'getting started guide', 'url': 'https://dev.drosera.io/trappers/getting-started'}, {'type': 'installation guide', 'url': 'https://dev.drosera.io/operators/installation'}, {'type': 'GitHub repository', 'url': 'https://github.com/drosera-network'}, {'type': 'examples repository', 'url': 'https://github.com/drosera-network/examples'}]}


'You are looking at a blockchain protocol called: Drosera\nHere are the contents of its developer docs and other relevant pages; use this information to build a short brochure of the protocol in markdown.\nLanding page:\nWebpage Title:\nDrosera · Ethereum Evolved – Docs\nWebpage Contents:\nSkip to content\nSearch...\nDocs\nExamples\n1.0.0\nDocs\nExamples\n1.0.0\nMenu\nDrosera · Ethereum Evolved\nDrosera\nInfinite bandwidth for smart contracts using\ndecentralized\n,\nverifiable\n, and\ncomposable\ninfrastructure\nIntroduction\nGitHub\nTerminal\nbash\nCopy\ncurl\n-L\nhttps://app.drosera.io/install\n|\nbash\nStart Trapping\nCreating traps with the\nDrosera CLI\nRunning A Node\nSpinning up a Drosera operator\nLitepaper\nHigh level drosera protocol and architecture\nUse Cases\nNovel ideas enabled by Traps and the drosera protocol\nCommon Actions\nBlockNumberTrap.sol\nTrapperCommands.sh\nOperatorCommands.sh\nFile\nBlockNumberTrap.sol\nCopy\n// SPDX-License-Identifier: UNLICENSED\npragma sol

In [72]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [61]:
create_brochure("Drosera", site_url)

Found links: {'links': [{'type': 'introduction page', 'url': 'https://dev.drosera.io/introduction'}, {'type': 'litepaper page', 'url': 'https://dev.drosera.io/litepaper'}, {'type': 'use cases page', 'url': 'https://dev.drosera.io/use-cases'}, {'type': 'GitHub examples', 'url': 'https://github.com/drosera-network/examples'}, {'type': 'GitHub repository', 'url': 'https://github.com/drosera-network'}]}


# Welcome to Drosera – The Future of Ethereum

Drosera is not just a protocol; it's a revolution in smart contract infrastructure. Get ready to elevate your DeFi game with unmatched capabilities designed for developers, investors, and visionaries eager to dominate the blockchain landscape.

## 🚀 Unique Design: Infinite Bandwidth for Smart Contracts

Drosera redefines what's possible on Ethereum by introducing **Traps**—dynamic smart contracts that trap essential on-chain data and unleash powerful on-chain responses. Say goodbye to limitations and embrace infinite bandwidth powered by decentralized architecture.

## 🌍 Addressing Market Necessities

In a world where vulnerability is a constant threat, Drosera steps in to offer unparalleled security. Born to detect exploits and mitigate financial losses, it simplifies creating automated monitoring systems, fortifying the DeFi ecosystem. We make sure your investments and protocols are always protected!

## 🛠 High-Level Technical Architecture

Drosera is built with cutting-edge technologies that include:

- **Drosera Traps:** Off-chain smart contracts that monitor critical invariants while responding rapidly to keep your assets secure.
- **RiscZero zk-Proofs:** This verifiable infrastructure guarantees your data is safe and sound.
- **Asynchronous Composability:** Seamless integration of off-chain computation with on-chain states using Solidity.
- **Testable Infrastructure:** Rally around Ethereum tooling with a developer-friendly environment inspired by Terraform.
- **Decentralized Nodes:** Powered by LibP2P for robust peer-to-peer networking, ensuring your contributions integrate smoothly for a composed society.

## 📊 Use Cases: Endless Possibilities to Dominate

Seize control with Drosera's Traps, adaptable to numerous DeFi contexts:

- **Automated Liquidation:** React instantly when collateralization ratios hit critical levels, protecting users and profits.
- **Market Analysis:** Capture pricing, liquidity changes, and trading volumes to formulate more effective strategies.
- **Cross-Chain Security:** Stay ahead by actively monitoring asset movements and finality across multiple chains.

From decentralized exchanges to insurance protocols, Drosera is the ultimate toolkit for anyone wanting to thrive in the crypto arena.✊

## 👨‍💻 Development Flow: Power in Your Hands

1. **Create a Trap:** Use the Drosera CLI to define your incident response protocols with precision.
2. **Run Operator Nodes:** Execute your Traps seamlessly, gaining permission for on-chain actions that maximize rewards.
3. **Monitor Feedback:** Leverage metrics and test your Traps through existing Ethereum tooling, ensuring reliability.

Ready to take on the challenge? Join the elite ranks of the crypto innovators!

## ✅ Join Us Today

The future is yours for the taking with Drosera. Join our **Telegram**, hop into our **Discord**, or visit our **GitHub** for code samples, examples, and the ever-evolving **Litepaper**. 

### **Start Trapping or Get Left Behind**

Drosera is here to empower you. Unleash your potential with endless possibilities—let’s dominate Ethereum together!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [73]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [74]:
stream_brochure("Drosera", site_url)

Found links: {'links': [{'type': 'introduction page', 'url': 'https://dev.drosera.io/introduction'}, {'type': 'litepaper', 'url': 'https://dev.drosera.io/litepaper'}, {'type': 'use cases page', 'url': 'https://dev.drosera.io/use-cases'}, {'type': 'getting started guide', 'url': 'https://dev.drosera.io/trappers/getting-started'}, {'type': 'installation guide', 'url': 'https://dev.drosera.io/operators/installation'}, {'type': 'GitHub repository', 'url': 'https://github.com/drosera-network'}, {'type': 'GitHub examples', 'url': 'https://github.com/drosera-network/examples'}]}


# Unleash the Power of Drosera: Ethereum Evolved 🔥

---

Welcome to **Drosera**, the next-gen protocol that’s tossing the old Ethereum model on its head. If you’re sick of mediocre systems that barely keep up with the explosive DeFi scene, listen up. Drosera is here to give you **infinite bandwidth** for smart contracts, seamlessly blending **decentralization**, **verifiability**, and **composability** into a protocol that’s as powerful as it is user-friendly. Let’s break it down.

---

## **Why Drosera? The Necessity in the Market**

DeFi is a wild west, and if your monitoring systems are outdated, you’re begging for trouble. Drosera emerged from the chaos, born out of the need for a **robust security framework** that can handle the heat of today’s financial landscape. We exist to empower developers and users to **detect exploits** and **mitigate financial losses**, all while simplifying how decentralized applications can react to events in real-time.

---

## **High-Level Technical Design 🛠️**

### **Traps - The Heart of Drosera**
Traps are where the magic happens. They are smart contracts tasked with gathering on-chain data and executing automated responses. Think of them as your watchdogs in the Ethereum jungle—monitoring like a hawk, reacting faster than your enemies can strike. 

- **On-chain and off-chain components:** Deliver timely responses to suit real-world demands.
- **Asynchronous composability:** This ensures flawless interactions between off-chain computation and on-chain state—yup, no hiccups.

### **Operators**
Drosera operators are not just users; they are the **guardians** of the network. The folks running these nodes use our powerful client software to monitor conditions and act as the first line of defense against threats. If you thought being an operator was just for tech nerds, think again. Only the truly elite will thrive.

### **Seed Nodes**
These bad boys host the Trap bytecode and help onboard operators into our decentralized network. But beware—only use trusted Seed Nodes, or you’ll risk compromising the integrity of your operations.

---

## **Use Cases: The Infinite Possibilities 🌟**

Here are just a few ways you can harness the og strength of Drosera:

- **Automated Liquidation Responses:** Never let collateralization ratios drop the ball again.
- **Data Integrity Checks:** Keep your oracles in check. Catch fraud before it happens!
- **Governance Mechanisms:** Safeguard against malicious actors trying to manipulate votes. You think you can just roll up and take over? Not on Drosera’s watch.

And that’s just scratching the surface. If you find a problem within DeFi, there's a high chance Drosera can tackle it.

---

## **Development Flow: Get Started Fast 🚀**

1. **Install the Drosera CLI:** Jump in quickly by executing this command:
   bash
   curl -L https://app.drosera.io/install | bash
   
2. **Create your Trap:** Use the provided templates and customize them according to your own killer idea.
3. **Run your Operator:** Opt into your chosen Trap and start executing commands that will leave the competition trembling.

With simple engagement strategies and a mighty backend, your path to domination is paved.

---

## **Join the Movement**

The Drosera community is not just an afterthought; it's **pioneering** the future of DeFi technology. Connect with us through our Discord, GitHub, or follow us on X to stay on the cutting edge of what's next. 

---

If you’re ready to escape the trap of outdated systems and join the elite ranks of DeFi innovators, then step up. Drosera isn’t just another protocol—it's a revolution. Are you in or out? 💪

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("Drosera", site_url)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>